Дана статистика пользователей adult.csv.  
Получите значения AUC для различных моделей и их параметров.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score

import time 

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('adult.csv')

In [3]:
df.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
age                48842 non-null int64
workclass          48842 non-null object
fnlwgt             48842 non-null int64
education          48842 non-null object
educational-num    48842 non-null int64
marital-status     48842 non-null object
occupation         48842 non-null object
relationship       48842 non-null object
race               48842 non-null object
gender             48842 non-null object
capital-gain       48842 non-null int64
capital-loss       48842 non-null int64
hours-per-week     48842 non-null int64
native-country     48842 non-null object
income             48842 non-null object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [5]:
def make_features_cat(df, cat_features):
    for each in cat_features:
        dummies = pd.get_dummies(df[each], prefix=each, drop_first=False)
        df = pd.concat([df, dummies], axis=1)
    return df

Значения по стране рождения сильно скошены в сторону USA, поэтому просто введем флаг is_from_usa 

In [6]:
df['is_from_usa'] = df.apply(lambda x : 1 if x['native-country'] == 'United-States' else 0, axis=1)
df = df.drop(labels='native-country', axis=1)
df['is_from_usa'].value_counts(normalize=True)

1    0.897424
0    0.102576
Name: is_from_usa, dtype: float64

In [7]:
df.income = df.income.map({'<=50K' : 0, '>50K' : 1})
df.income.value_counts()

0    37155
1    11687
Name: income, dtype: int64

In [8]:
df['is_male'] = df.gender.map({'Female' : 0, 'Male' : 1})
df = df.drop(labels='gender', axis=1)
df.is_male.value_counts()

1    32650
0    16192
Name: is_male, dtype: int64

У нас есть два близких признака: education и educational-num. Второй является по сути LabelEncoding для первого, поэтому уберем education

In [9]:
df = df.drop(labels='education', axis=1)

In [10]:
categorical_features = ['workclass', 'marital-status', 'occupation', 'relationship', 'race']
numeric_features =  ['fnlwgt', 'capital-gain', 'capital-loss']

df = make_features_cat(df, categorical_features)
df = df.drop(labels=categorical_features, axis=1)

In [11]:
y = df.income
df = df.drop('income', axis=1)
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)

In [12]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_train = pd.DataFrame(X_train, columns=df.columns)

X_test = scaler.transform(X_test)
X_test = pd.DataFrame(X_test, columns=df.columns)

X_train.head()

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week,is_from_usa,is_male,workclass_?,workclass_Federal-gov,...,relationship_Not-in-family,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,race_Other,race_White
0,0.246123,-0.421839,-0.029836,-0.144456,-0.219726,-0.034370,0.337333,0.703432,-0.248005,-0.173749,...,-0.587862,-0.177543,-0.428276,-0.341273,-0.222493,-0.098837,-0.180512,-0.321871,-0.09087,0.408979
1,0.974794,-0.138604,-1.585248,-0.144456,-0.219726,-0.435727,0.337333,0.703432,-0.248005,-0.173749,...,-0.587862,-0.177543,-0.428276,-0.341273,-0.222493,-0.098837,-0.180512,-0.321871,-0.09087,0.408979
2,-0.336813,-0.161041,-0.418689,-0.144456,-0.219726,-0.034370,0.337333,0.703432,-0.248005,-0.173749,...,-0.587862,-0.177543,-0.428276,-0.341273,-0.222493,-0.098837,-0.180512,-0.321871,-0.09087,0.408979
3,-0.774016,-0.605742,-0.418689,-0.144456,-0.219726,-0.034370,0.337333,-1.421602,-0.248005,-0.173749,...,1.701080,-0.177543,-0.428276,-0.341273,-0.222493,-0.098837,-0.180512,-0.321871,-0.09087,0.408979
4,0.537592,-1.529544,0.359017,-0.144456,-0.219726,0.768345,0.337333,0.703432,-0.248005,-0.173749,...,1.701080,-0.177543,-0.428276,-0.341273,-0.222493,-0.098837,-0.180512,-0.321871,-0.09087,0.408979


Обучим модель в лоб и посмотрим на качество

In [13]:
model = LogisticRegression(C = 0.1)
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.8559729757395844

In [14]:
roc_auc_score(model.predict(X_test), y_test)

0.8095003668043416

Использование модели в лоб дало низкое качество. Теперь поиграемся с параметрами и моделями.  
В качестве моделей возьмем логистическую регрессию, случайный лес, knn и gbm.  
Проверять будем по метрике ROC AUC с использованием GridSearch и Cross Validation.  
Тестовую выборку будем использовать в качестве валидационной.

#### Логистическая регрессия

In [15]:
clf_logreg = LogisticRegression()
logreg_params = {'C': [ 10 ** x for x in range(-2, 3, 1) ]
                ,'penalty' : [ 'l1', 'l2' ]
                }

#### RandomForest

In [16]:
clf_rf = RandomForestClassifier()
rf_params = {'n_estimators': [ 30, 50, 100, 300 ]
            ,'max_depth': [ 5, 10, 15]
            }

#### KNN

In [17]:
clf_knn = KNeighborsClassifier()
knn_params = { 'n_neighbors' : [ 20 ]
             , 'weights' : ['uniform', 'distance']
             }

#### GBM

In [18]:
clf_gbm = GradientBoostingClassifier()
gbm_params = { 'n_estimators' : [150 , 200]
              , 'loss' : ['deviance']
              , 'max_depth' : [5, 7]
             } 

Функция для расчета моделей

In [20]:
def calculate_models(models, cv=10, scoring='roc_auc'):
    for model in models:
        start = time.time()
        print('-----------------------------------------------------------')
        print('Model : {}'.format(model[0].__class__.__name__))
        print('Parameters : {}'.format(model[1]))
        gs = GridSearchCV( model[0], param_grid = model[1], cv=cv, scoring=scoring, n_jobs=-1 )
        gs.fit(X_train, y_train)
        print('Best params      : {}'.format(gs.best_params_))
        print('Train score      : {}'.format(gs.best_score_))
        print('Validation score : {}'.format(gs.score(X_test, y_test)))
        print('Calculation time : {} sec'.format(time.time() - start))

In [21]:
models = [ (clf_logreg, logreg_params)
          ,(clf_rf, rf_params)
          ,(clf_knn, knn_params)
          ,(clf_gbm, gbm_params)
         ]

calculate_models(models=models, cv=3, scoring='roc_auc')

-----------------------------------------------------------
Model : LogisticRegression
Parameters : {'C': [0.01, 0.1, 1, 10, 100], 'penalty': ['l1', 'l2']}
Best params      : {'C': 0.1, 'penalty': 'l1'}
Train score      : 0.9053107091402491
Validation score : 0.9097917838068864
Calculation time : 26.740007877349854 sec
-----------------------------------------------------------
Model : RandomForestClassifier
Parameters : {'n_estimators': [30, 50, 100, 300], 'max_depth': [5, 10, 15]}
Best params      : {'max_depth': 15, 'n_estimators': 100}
Train score      : 0.917470872232082
Validation score : 0.9229983692329788
Calculation time : 41.2983615398407 sec
-----------------------------------------------------------
Model : KNeighborsClassifier
Parameters : {'n_neighbors': [20], 'weights': ['uniform', 'distance']}
Best params      : {'n_neighbors': 20, 'weights': 'distance'}
Train score      : 0.8763804482012499
Validation score : 0.8835468861575146
Calculation time : 240.52824187278748 sec